In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/Kats/main/kats/data/air_passengers.csv
!wget https://raw.githubusercontent.com/facebookresearch/Kats/main/kats/data/multi_ts.csv

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings('ignore')

from kats.consts import TimeSeriesData

In [ ]:
try: # If running on Jupyter
    air_passengers_df = pd.read_csv("../kats/data/air_passengers.csv")
except FileNotFoundError: # If running on colab
    air_passengers_df = pd.read_csv("air_passengers.csv")

"""
Note: If the column holding the time values is not called "time", you will want to specify 
the name of this column using the time_col_name parameter in the TimeSeriesData constructor.
"""
air_passengers_df.columns = ["time", "value"]
air_passengers_df.head()

In [ ]:
multi_ts_df = pd.read_csv("datasets/wit-10hz.csv", sep='\t')
multi_ts_df.columns = ["time", "acx", "acy", "acz", "gyx", "gyy", "gyz", "roll", "pitch", "yaw"]
multi_ts_df.head()

In [ ]:
# air_passengers_ts = TimeSeriesData(air_passengers_df)
multi_ts = TimeSeriesData(multi_ts_df)

In [ ]:
# check that the type of the data is a "TimeSeriesData" object for both cases
# print(type(air_passengers_ts))
print(type(multi_ts))

In [ ]:
# For the air_passengers TimeSeriesData, check that both time and value are pd.Series
print(type(air_passengers_ts.time))
print(type(air_passengers_ts.value))

In [ ]:
# For the multi_ts TimeSeriesData, time is a pd.Series and value is a pd.DataFrame
print(type(multi_ts.time))
print(type(multi_ts.acx))

In [ ]:
# air_passengers_ts_from_series = TimeSeriesData(time=air_passengers_df.time, value=air_passengers_df.value)
multi_ts_from_series = TimeSeriesData(time=multi_ts_df.time, value=multi_ts_df[['acx','acy','acz','gyx','gyy','gyz','roll','pitch','yaw']])

In [ ]:
from dateutil import parser
from datetime import datetime

# Convert time from air_passengers data to unix time
air_passengers_ts_unixtime = air_passengers_df.time.apply(
    lambda x: datetime.timestamp(parser.parse(x))
)

air_passengers_ts_unixtime

In [ ]:
# Create the TimeSeriesData object with the extra arguments to accomodate unix time 
ts_from_unixtime = TimeSeriesData(
        time=air_passengers_ts_unixtime, 
        value=air_passengers_df.value, 
        use_unix_time=True, 
        unix_time_units="s"
)

ts_from_unixtime

In [ ]:
air_passengers_ts[1:5]

In [ ]:
air_passengers_ts[1:5] + air_passengers_ts[1:5]

In [ ]:
# Equality and Inequality are also supported:

air_passengers_ts == air_passengers_ts_from_series

In [ ]:
multi_ts == multi_ts_from_series

In [ ]:
# length

len(air_passengers_ts)

In [ ]:
# Creating two slices
ts_1 = air_passengers_ts[0:3]
ts_2 = air_passengers_ts[3:7]

ts_1.extend(ts_2)
ts_1

In [ ]:
%matplotlib inline

# Must pass the name of the value columns to plot
air_passengers_ts.plot(cols=['value'])
plt.show()

In [ ]:
# We can plot multiple time series from multi_ts by passing in the name of each value column we want to plot
multi_ts.plot(cols=['acz'])
plt.show()

In [ ]:
# Initiate feature extraction class
from kats.tsfeatures.tsfeatures import TsFeatures
tsFeatures = TsFeatures()

In [ ]:
# features_air_passengers = TsFeatures().transform(air_passengers_ts)
features_ts = TsFeatures().transform(multi_ts)

In [ ]:
# features_air_passengers
# features_air_passengers
features_ts

In [ ]:
import os
from cmdstanpy import cmdstan_path, CmdStanModel

# specify locations of Stan program file and data
stan_file = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.stan')
data_file = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.data.json')

# instantiate a model; compiles the Stan program by default
model = CmdStanModel(stan_file=stan_file)

# obtain a posterior sample from the model conditioned on the data
fit = model.sample(chains=4, data=data_file)

# summarize the results (wraps CmdStan `bin/stansummary`):
fit.summary()